# Project description

You're an analyst at Crankshaft List. Hundreds of free advertisements for used vehicles are published on your site every day. You need to study data collected over the last few years and determine which factors influence the price of a vehicle.
Instructions for completing the project

## Step 1. Open the data file and study the general information
File path: /datasets/vehicles_us.csv. Download dataset

## Step 2. Data preprocessing
*  Identify and study missing values:
    * In some cases there's an obvious way to replace missing values. For instance, if a Boolean field contains only True values, it's reasonable to assume that the missing values are False. There aren't such obvious fixes for other data types, and there are cases when the fact that a value is missing is significant. In such instances, don't fill in the values.
    * When appropriate, do fill in the values. Explain why you chose to do so and how you selected the replacement values.
    * Describe the factors that may have resulted in missing values.

* Convert the data to the required types:
Indicate the columns where the data types need to be changed and explain why.

## Step 3. Calculate and add to the table the following:
* Day of the week, month, and year the ad was placed
* The vehicle's age (in years) when the ad was placed
* The vehicle's average mileage per year

In the condition column, replace string values with a numeric scale:
* new = 5
* like new = 4
* excellent = 3
* good = 2
* fair = 1
* salvage = 0


## Step 4. Carry out exploratory data analysis, following the instructions below:
* Study the following parameters: price, vehicle's age when the ad was placed, mileage, number of cylinders, and condition. Plot histograms for each of these parameters. Study how outliers affect the form and readability of the histograms.
* Determine the upper limits of outliers, remove the outliers and store them in a separate DataFrame, and continue your work with the filtered data.
* Use the filtered data to plot new histograms. Compare them with the earlier histograms (the ones that included outliers). Draw conclusions for each histogram.
* Study how many days advertisements were displayed (days_listed). Plot a histogram. Calculate the mean and median. Describe the typical lifetime of an ad. Determine when ads were removed quickly, and when they were listed for an abnormally long time.
* Analyze the number of ads and the average price for each type of vehicle. Plot a graph showing the dependence of the number of ads on the vehicle type. Select the two types with the greatest number of ads.
* What factors impact the price most? Take each of the popular types you detected at the previous stage and study whether the price depends on age, mileage, condition, transmission type, and color. For categorical variables (transmission type and color), plot box-and-whisker charts, and create scatterplots for the rest. When analyzing categorical variables, note that the categories must have at least 50 ads; otherwise, their parameters won't be valid for analysis.

## Step 5. Write an overall conclusion
Format: Complete the task in a Jupyter notebook. Put your code in the code cells and your text explanations in markdown cells, then apply formatting and headings.


## Description of the data
The dataset contains the following fields:
* price
* model_year
* model
* condition
* cylinders
* fuel — gas, diesel, etc.
* odometer — the vehicle's mileage when the ad was published
* transmission
* paint_color
* is_4wd — whether the vehicle has 4-wheel drive (Boolean type)
* date_posted — the date the ad was published
* days_listed — from publication to removal


## Initialization

Let's import all relevant libraries

In [ ]:
# Loading all the libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


### Load data

Now, let's load our data and look at the general information.

In [ ]:
# Load the data file into a DataFrame
vehicles_us=pd.read_csv('/datasets/vehicles_us.csv')


### Explore initial data

The dataset contains the following fields:
- `price`
- `model_year`
- `model`
- `condition`
- `cylinders`
- `fuel` — gas, diesel, etc.
- `odometer` — the vehicle's mileage when the ad was published
- `transmission`
- `paint_color`
- `is_4wd` — whether the vehicle has 4-wheel drive (Boolean type)
- `date_posted` — the date the ad was published
- `days_listed` — from publication to removal



In [ ]:
# print the general/summary information about the DataFrame
vehicles_us.info()


In [ ]:
# print a sample of data
vehicles_us.head(10)


The data contains 51525 rows and 12 columns.

The values in `date posted` are object type values when they should be a date time type. 

We have missing values in several columns - `model_year`, `cylinders`, `odometer`, `paint_color` and `is_4wd`. The last one contains the most number of missing values.

### Conclusions and further steps


We will fill missing values in `model_year`, `cylinders` and `odometer` according to the most common value to each model type. For `paint_color`, we will fill with `unknown` and for `is_4wd`, we will change the missimg value with 0 because the data type is Boolean type. That means that if the value is 1, it is true and if it is missing than it is 0 - False.

## Treat missing values (if any)


We will check all missing values that we can treat.

In [ ]:
vehicles_us.isna().sum()

First, we will start filling the missing values int he `model_year` column. It is logocal to fill them accorging to the model type. But with what value should we choose - mean, median or the most common value?
Let's see the disctribution of those values.


In [ ]:
vehicles_us['model_year'].describe()

From the description, we see that up to 25% of all values, it is between `1908`-`2007`, a 100 year-old gape while between 25% to maximun value, it is spread between `2007`-`2019`. The discrituation is not symetricall but is skewed. Therefore, we cannot use the mean values but with the median or the most common values. Let's try to replace it with thhe most common value grouping it by the model type using `.mode()`.


In [ ]:
vehicles_us['model_year']=vehicles_us.groupby('model')['model_year'].transform(lambda grp: grp.fillna(grp.mode().max()))


In [ ]:
vehicles_us['model_year'].isna().sum()

Let's try applying the same logic to the `cylinders` column.

In [ ]:
vehicles_us['cylinders'].describe()

In this case, we cannot determine what is the detribution by using `.describe()`. Lets try with `.value_counts()`.


In [ ]:
vehicles_us['cylinders'].value_counts()

According to this destribution, the counts of values of `cylinders` starts with:
* low when it is equal to `3.0`
* then goes up if it is equal to `4.0`
* goes down if it is equal to `5.0`
* goes up if it is equal to `6.0` and `8.0`
* and finaly goes down if it is equal to `10.0` and `12.0`.
 
Therefore, we will be using this time the median value of cylinder according to tho `model`  using `.groupby()` and `.median()`.


In [ ]:
vehicles_us['cylinders']=vehicles_us['cylinders'].fillna(vehicles_us.groupby('model')['cylinders'].transform('median'))

In [ ]:
vehicles_us['cylinders'].isna().sum()

In [ ]:
vehicles_us['cylinders'].value_counts()

Now, let's try applying the same logic to the `odometer` column.

In [ ]:
vehicles_us['odometer'].describe()

From the description, we see that:
* up to 25% of all values, it is between `0`-`70000`
* between 25% to 75%, the values are distributed between `70000`-`15500`.
* and from 75% to maximun value, they are distrinuted between `15500`-`99000`.

The discrituation is not symetricall but is right skewed. Therefore, we cannot use the mean values but with the median values grouping the values according to the `model` and `condition`. Choosing the condition as one of the characteristic is due to the fact that if the model is in as new, the value if the odometer should be low as well as for a model who is in bad condition who should have a high value of odometer.


In [ ]:
vehicles_us['odometer']=vehicles_us['odometer'].fillna(vehicles_us.groupby('model_year')['odometer'].transform('median'))


In [ ]:
vehicles_us['odometer'].isna().sum()

In [ ]:
vehicles_us[vehicles_us['odometer'].isna()]

In [ ]:
vehicles_us.query('model_year<=1940')

We have no information for `odometer` values for `model_year` 1929 and the difference between the `odometer` values for models produced between 1908 to 1936 is bigger 5 times. Therefore, we will decide to drop this row. 

In [ ]:
vehicles_us=vehicles_us.dropna(subset=['odometer'])

In [ ]:
vehicles_us['odometer'].isna().sum()

Now let's check the `paint_color` column.

In [ ]:
vehicles_us['paint_color'].unique()

We cannot know according to which categoried to fill it. Therefor, we will replace the missing values in `unknown`.


In [ ]:
vehicles_us['paint_color']=vehicles_us['paint_color'].fillna('unknown')

In [ ]:
vehicles_us['paint_color'].isna().sum()

Finelly, let's check the `is_4wd` column. It looks like it have the most high missing values. 

In [ ]:
"{0:.0%}".format(len(vehicles_us[vehicles_us['is_4wd'].isna()])/len(vehicles_us))

In [ ]:
vehicles_us['is_4wd'].unique()

It looks like 50% of the values in the `is_4wd` is missing. But, in the project description, it is written that the value type in this column is Boolean. That mean if the value is `1.0` than it is True that the model car has a 4wd. If it is False than the model type has no 4wd. Therefore, we can conclude that the missing values inthis column represent the False value and we will replace it with `0.0`.


In [ ]:
vehicles_us['is_4wd']=vehicles_us['is_4wd'].fillna(0)

In [ ]:
vehicles_us['is_4wd'].unique()

In [ ]:
vehicles_us.isna().sum()

## Fix data types


Let's check again the data type for each column using `.info()`

In [ ]:
vehicles_us.info()


We have an issue the `days_posted`coulmn - it is an object and should a dateframe.
So our next steps will be replacing the `date_posted` to a dateFrame usind `.to_datetime()` function.


In [ ]:
vehicles_us['date_posted'][0]

In [ ]:
vehicles_us['date_posted']=pd.to_datetime(vehicles_us['date_posted'],format='%Y-%m-%d')

In [ ]:
vehicles_us.info()

## Enrich data


Now we will add new columns to the data:
* using `dt.weekday` for day of the week that the ad was placed in the new column `day_of_the_week`
* using `dt.month` The month that the ad was placed in the new column `month_posted`
* using `dt.year` The year that ad was placed.in the new column `year_posted`
* The vehicle's age (in years) when the ad was placed in the new column `vehicle_age`(substraction of the `model_year` from `year_posted`.
* The vehicle's average mileage per yearin the new column `avg_mils_per_year`(dividing the `odometer` by the `vehicle_age`).

In [ ]:
# Add datetime values for when the ad was placed
vehicles_us['day_of_the_week']=vehicles_us['date_posted'].dt.weekday
vehicles_us['month_posted']=vehicles_us['date_posted'].dt.month
vehicles_us['year_posted']=vehicles_us['date_posted'].dt.year
vehicles_us.head(5)

In [ ]:
# Add the vehicle's age when the ad was placed
vehicles_us['vehicle_age']=vehicles_us['year_posted']-vehicles_us['model_year']+1
vehicles_us['vehicle_age'].sort_values().unique()

To avoid a division by age equal to 0, we add +1 to the calculation of the `vehicle_age`.

In [ ]:
# Add the vehicle's average mileage per year
vehicles_us['avg_mils_per_year']=vehicles_us['odometer']/vehicles_us['vehicle_age']

In [ ]:
vehicles_us['avg_mils_per_year'].unique()

Finaly, let's simplify the `condition` column by creating a dictionay `condition_change` of condition values as keys and their values as following:
* new = 5
* like new = 4
* excellent = 3
* good = 2
* fair = 1
* salvage = 0

And then changing the values in the `condition` column by the values in dictionary using `.map()` method.

In [ ]:
# It might help to replace the condition values with something that can be manipulated more easily
condition_change={
    'new':5,
    'like new' : 4,
    'excellent' : 3,
    'good' : 2,
    'fair' : 1,
    'salvage' : 0,
}


In [ ]:
vehicles_us['condition']=vehicles_us['condition'].map(condition_change)

In [ ]:
vehicles_us['condition'].value_counts()

## Check clean data



Let's take a look at the preprocessed data by printing the general information again with `.info()` and the 10 first rows using `.head()`.


In [ ]:
# print the general/summary information about the DataFrame

vehicles_us.info()

In [ ]:
# print a sample of data
vehicles_us.head(10)


## Study core parameters


After the data ws preproccessed, let's take a quick look at the ploted histograms for the following parameters:
*  price
* vehicle's age
* mileage
* cylinders
* condition

To not repeat which representation, we will creat a function `parameter_distribution` which will be applied to each parameter and will represent the following:
* the plot
* of the plot is skewed
* the discription of the parameter
* the Q1, Q3, IQR, minumum and maximun outeliers values

[If you find that you need to repeat the same jobs multiple times, think about what coding instrument might be helpful for such a thing to avoid duplicating your code.]

In [ ]:
#import the plot library.

import matplotlib.pyplot as plt


In [ ]:
def parameter_distribution(parameter):
    print('The histogram of: {}'.format(parameter))
    vehicles_us[parameter].hist()
    plt.show()
    if vehicles_us[parameter].mean()>vehicles_us[parameter].median():
        print('The distribution is skewed to the right')
    else:
        print('The distribution is skewed to the left')
    print('---------------------------------------')
    print('Statistics on: {}'.format(parameter))
    print(vehicles_us[parameter].describe())
    Q1 = vehicles_us[parameter].quantile(0.25)
    Q3 = vehicles_us[parameter].quantile(0.75)
    IQR = Q3 - Q1
    min_iqr=Q1-1.5*IQR
    max_iqr=Q3+1.5*IQR
    print('---------------------------------------')
    print('min border:',min_iqr)
    print('max border:',max_iqr)
    print('---------------------------------------')
    if min_iqr<0:
        print('The outliers are considered to be values above',max_iqr)
        print('We have',len(vehicles_us[vehicles_us[parameter]>max_iqr]),"values that we can consider outliers")
    else:
        print('The outliers are considered to be values below',min_iqr, "and above",max_iqr)
        print('We have',len(vehicles_us[(vehicles_us[parameter]<min_iqr)|(vehicles_us[parameter]>max_iqr)]),"values that we can consider outliers")
    print()

In [ ]:
#
for parameter in ['price','vehicle_age','avg_mils_per_year','cylinders','condition']:
    parameter_distribution(parameter)



## Study and treat outliers


Given the results above, we can see that we have outliers in all parameters selected. Mostley the minimun border is negative while the minimun values starts with 0 except in the `cylinders`. 

We will build two tables for the lower and upper borders which we will use to slic our data.

In [ ]:
# Determine the lower limits for outliers
lower_outliers=[]
for parameter in ['price','vehicle_age','avg_mils_per_year','cylinders','condition']:
    min_iqr=vehicles_us[parameter].quantile(0.25)-1.5*(vehicles_us[parameter].quantile(0.75)-vehicles_us[parameter].quantile(0.25))
    lower_outliers.append([parameter,min_iqr])


In [ ]:
lower_outliers

In [ ]:
lower_outliers=pd.DataFrame(lower_outliers,columns=['parameter','lower_outliers'])
lower_outliers

In [ ]:
# Determine the upper limits for outliers
upper_outliers=[]
for parameter in ['price','vehicle_age','avg_mils_per_year','cylinders','condition']:
    max_iqr=vehicles_us[parameter].quantile(0.75)+1.5*(vehicles_us[parameter].quantile(0.75)-vehicles_us[parameter].quantile(0.25))
    upper_outliers.append([parameter,max_iqr])


In [ ]:
upper_outliers

In [ ]:
upper_outliers=pd.DataFrame(upper_outliers,columns=['parameter','upper_outliers'])
upper_outliers

From the lower and upper outliers, we will use the following to slice our data:
* `condition` which are not equal to 0 and lower than 4.5.
* `price` is lower and equal to 34595.5.
* `vehicle_age` is lower and equal to 24.
* `avg_mils_per_year` is lower and equa to 40368
* and `cylinders` are lower than 14.

In [ ]:
# Store data without outliers in a separate DataFrame

good_vehicle_df=vehicles_us.query('condition !=0 & price<=34597.5 & vehicle_age<=24 & avg_mils_per_year<=40368 & cylinders<=14 & condition <= 4.5')

good_vehicle_df

## Study core parameters without outliers


Using the filtered data `good_vehicle_df`,  first we will compare old ploting of raw data compared to the filtered and then, we will plot separts and clear histograms with the same parameters `price`,`vehicle_age`,`avg_mils_per_year`,`cylinders` and `condition`. 

In [ ]:
vehicles_us['price'].hist(bins=30,figsize=(15,10))
good_vehicle_df['price'].hist(bins=30,figsize=(15,10))
plt.show()
vehicles_us['vehicle_age'].hist(bins=30,figsize=(15,10))
good_vehicle_df['vehicle_age'].hist(bins=30,figsize=(15,10))
plt.show()
vehicles_us['avg_mils_per_year'].hist(bins=30,figsize=(15,10))
good_vehicle_df['avg_mils_per_year'].hist(bins=30,figsize=(15,10))
plt.show()
vehicles_us['cylinders'].hist(bins=30,figsize=(15,10))
good_vehicle_df['cylinders'].hist(bins=30,figsize=(15,10))
plt.show()
vehicles_us['condition'].hist(bins=30,figsize=(15,10))
good_vehicle_df['condition'].hist(bins=30,figsize=(15,10))


In [ ]:
good_vehicle_df[['price','vehicle_age','avg_mils_per_year','cylinders','condition']].hist(bins=30,figsize=(15,10))


We can conclude that:
* We can find more cars that their price is less than 10000.
* Most ads were place on cars that hace less than 10 year old. 
* The most commun average miles per years is betwen 10000-15000.
* Most commun cylinder is 6, then 8 and 4.
* moost cars are in excellent or good condition.

## Ads lifetime



Now, let's chech the distibution of `days_listed` of the advertisements using the same method previously to investigate the parameters.

In [ ]:
good_vehicle_df['days_listed'].describe()

In [ ]:
print('The histogram of: {}'.format('days_listed'))
good_vehicle_df['days_listed'].hist()
plt.show()
if good_vehicle_df['days_listed'].mean()>good_vehicle_df['days_listed'].median():
    print('The distribution is skewed to the right')
else:
    print('The distribution is skewed to the left')
print('---------------------------------------')
print('Statistics on: {}'.format(parameter))
print(good_vehicle_df['days_listed'].describe())
Q1 = good_vehicle_df['days_listed'].quantile(0.25)
Q3 = good_vehicle_df['days_listed'].quantile(0.75)
IQR = Q3 - Q1
min_iqr=Q1-1.5*IQR
max_iqr=Q3+1.5*IQR
print('---------------------------------------')
print('min border:',min_iqr)
print('max border:',max_iqr)
print('---------------------------------------')
if min_iqr<0:
    print('The outliers are considered to be values above',max_iqr)
    print('We have',len(good_vehicle_df[good_vehicle_df['days_listed']>max_iqr]),"values that we can consider outliers")
else:
    print('The outliers are considered to be values below',min_iqr, "and above",max_iqr)
    print('We have',len(good_vehicle_df[(good_vehicle_df['days_listed']<min_iqr)|(good_vehicle_df['days_listed']>max_iqr)]),"values that we can consider outliers")
print()

We can conclude that most ads were place for at least 50 days.

## Average price per each type of vehicle

We will creat a pivot table for types with the highest number of ads and show their average price and represent it in a bar type plot.

In [ ]:
price_per_type=good_vehicle_df.pivot_table(index='type',values='price',aggfunc=('count','mean')).sort_values('count',ascending=False)
price_per_type


In [ ]:
price_per_type.plot(title='count of ads per type',kind='bar',grid=True, figsize=(15,10))

From the table above `price_per_type`, the top two type cars that have the most advertisements are:
* `Sedan` 
* `SUV` 

## Price factors


Depending on the research made above, we will investigate fo`Ford f-150` and `Chevrolet silverado 1500` the following factors that can impact on the price:
* age
* mileage
* condition
* transmission type
* and color.

For numeric variables in `vehicle_age`, `avg_mils_per_year` and `condition`, we will plot a scatter representation and for categorical variables such as `transmission` and `paint_color`, we will represent in a box-and-whisker plot. Then investigate their coloration.

Both model types have more than 50 ads***

In [ ]:
sedan=good_vehicle_df.query('type=="sedan"')
sedan


In [ ]:
sedan_scatter=sedan[['price','vehicle_age','avg_mils_per_year','condition']]
sedan_scatter

In [ ]:
sns.pairplot(sedan_scatter)

In [ ]:
sedan_scatter.corr()

In [ ]:
plt.figure(figsize=(15,10))
ax=sns.boxplot(x="transmission", y="price", data=sedan,showfliers = False)
print(sedan['transmission'].value_counts())
plt.show()
plt.figure(figsize=(15,10))
ax=sns.boxplot(x="paint_color", y="price", data=sedan,showfliers = False)
print(sedan['paint_color'].value_counts())

In [ ]:
suv=good_vehicle_df.query('type=="SUV"')
suv

In [ ]:
suv_for_scatter=suv[['price','vehicle_age','avg_mils_per_year','condition']]
suv_for_scatter

In [ ]:
sns.pairplot(suv_for_scatter)

In [ ]:
suv_for_scatter.corr()


In [ ]:
plt.figure(figsize=(15,10))
ax=sns.boxplot(x="transmission", y="price", data=suv,showfliers = False)
print(suv['transmission'].value_counts())
plt.show()
plt.figure(figsize=(15,10))
ax=sns.boxplot(x="paint_color", y="price", data=suv,showfliers = False)
print(suv['paint_color'].value_counts())

## General conclusion

[List your important conclusions in this final section, make sure they include those that led you to the way you processed and analyzed the data. Cover the missing values, duplicates, outliers and possible reasons and solutions for problematic artifacts that you had to address. Don't forget to remove all bracketed comments before submitting your project.]

We filled missing values in `model_year`and `cylinders` according to the most common value to each model type. \
For `odometer` missing value, we replaced according to the `model_year` and droped one row that had no odometer valur for year 1929.\
For `paint_color`, we filled with `unknown` and for `is_4wd` changed the missimg value with 0 because the data type is Boolean type. That means that if the value is 1, it is true and if it is missing than it is 0 - False.\
We had no duplicates.\
we changed the varibles in `date_posted` to a datetime type.\
we added new columns - `day_of_the_week`,`month_posted`, `year_posted`, `vehicle_age`and `avg_mils_per_year`.\
We changed the categorial variables in `condition`  column to numeric variables.\
We discovered lower outerlier in `condition` column and upper outleiers in `price`, `vehicle_age`, `avg_mils_per_year`, `cylinders`and `condition`. And according to this outliers, we filtered our data with which we will drow the following conclusions:
* We can find more cars that their price is less than 10000.
* Most ads were place on cars that have less than 10 year old.
* The most commun average miles per years is betwen 10000-15000.
* Most commun cylinder is 6, then 8 and 4.
* Most cars are in excellent or good condition.
* Ads have 50 days of life time.
* Top 2 model type that have the most ads are:
    * Sedan
    * SUV

Comparing in top 2 model types `sedan` and `SUV`, we founf the following conclusion that impact on the price:
* As the age does high, the price of the model goes down.
* When the average miles per years and the condition of the car have high values, the price will be raising.
* automatic sedan is higher than manual sedan. But, it's the opposite for the SUV. We can imagine that it can be due to the spare parts of the transmission.
* Orange sedan and black sedan have higher prices. It can be due to custom color dmand or most common color demand.
